In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.imagenet_utils import decode_predictions
from keras import layers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"/content/drive/MyDrive/viziometrics/train",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"/content/drive/MyDrive/viziometrics/val",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"/content/drive/MyDrive/viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2743 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1593 images belonging to 5 classes.



We will use a pretrained network (ResNet50) to extract features from the images. Then we will train a not-so-deep neural network on the features (X) and the labels (y). Finally we will evaluate our network.

In [ ]:
# download the pre-trained ResNet50 model
resnet = keras.applications.resnet.ResNet50(weights='imagenet', input_shape=(224,224,3))

# inspect the ResNet50 architecture
resnet.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
# The "embedding layer" is the "avg_pool" layer, the layer before the predictions.
# We want the outputs from this layer

layer_name = 'avg_pool'
resnet_embedder = Model(inputs=resnet.input,outputs=resnet.get_layer(layer_name).output)

# observe that this layer has output shape 2048
# let's save this dimension for later use
DIM = 2048

Using ResNet50 to extract features from the images. Printing the shape of the extracted features.

We use [`model.predict()`](https://keras.io/models/sequential/#predict_generator) to extract features. This function will return the output of the last layer of ResNet50, namely, conv5_block3_out (Activation). These are the features captured by the pretrained network.


In [ ]:
# use ResNet50 to extract features from the images
train_features = resnet_embedder.predict(train_generator, verbose=1)
val_features = resnet_embedder.predict(val_generator, verbose=1)
test_features = resnet_embedder.predict(test_generator, verbose=1)

print(train_features.shape, val_features.shape, test_features.shape)

1593/1593 [==============================] - 1223s 767ms/step
(2743, 2048) (1571, 2048) (1593, 2048)


We want to reshape the features into 2D arrays, which will be the input X of our not-so-deep neural network classifier. Running the following code and print the shape of the 2D arrays.

In [ ]:
# reshape the features to 2D arrays
train_X = train_features.reshape((-1, DIM))
val_X = val_features.reshape((-1, DIM))
test_X = test_features.reshape((-1, DIM))

print(train_X.shape, val_X.shape, test_X.shape)

(2743, 2048) (1571, 2048) (1593, 2048)


Reading the class labels from the generators. Printing `test_labels`

In [ ]:
# read the class labels from the generators
train_labels = train_generator.classes
val_labels = val_generator.classes
test_labels = test_generator.classes

print(test_labels)

[0 0 0 ... 4 4 4]


Observed that the labels are not one-hot encoded. We need to one-hot encode these labels and they will be the y that our model will predict.

In [ ]:
# get one-hot encoding of labels
def get_one_hot(labels, nb_classes):
    res = np.eye(nb_classes)[np.array(labels).reshape(-1)]
    return res.reshape(list(labels.shape)+[nb_classes])

# use get_one_hot()
NUM_CLASSES = 5
train_y = get_one_hot(train_labels, NUM_CLASSES)
val_y = get_one_hot(val_labels, NUM_CLASSES)
test_y = get_one_hot(test_labels, NUM_CLASSES)

Defining a neural network that contains a few layers.

In [ ]:
# create our model: a not-so-deep neural network
model = Sequential()

# Input layer takes arrays of shape (*, DIM)
model.add(layers.Dense(500, activation="relu", input_shape=(DIM,)))
# Added a hidden layer
model.add(layers.Dense(250, activation="relu"))
# Added a dropout layer for regularization
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
# Added another hidden layer
model.add(layers.Dense(200, activation="relu"))
# Output layer with NUM_CLASSES classes and softmax activation
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

# print out network architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               1024500   
                                                                 
 dense_1 (Dense)             (None, 250)               125250    
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               50200     
                                                                 
 dense_3 (Dense)             (None, 5)                 1005      
                                                                 
Total params: 1,200,955
Trainable params: 1,200,955
Non-trainable params: 0
_________________________________________________________________


Compiled our model using [`model.compile()`](https://keras.io/models/model/#compile). Choose `sgd` optimizer and `categorical_crossentropy` loss. Set `metrics = ["accuracy"]`

In [ ]:
# you need to compile the model before you can train it
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

Trained the model we defined for about 10-20 epochs. The model is trained on `train_X` and `train_y`; it is validated on `val_X` and `val_y`. The train accuracy and validation accuracy changes over epochs. Reporting the best train and validation accuracy we see during training.

In [ ]:
# save model weights while model is under training
checkpoint_path = '/content/drive/MyDrive/IMT 575/model_checkpoint.h5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', mode='max', save_best_only=True)

# train the model for about 20 epochs
# each epoch takes about 20 seconds on my laptop
model.fit(
    x = train_X,
    y = train_y,
    epochs = 20,
    validation_data = (val_X,val_y),
    callbacks = [checkpoint]
)
# The best train and validation accuracy during training is 0.9697 and 0.77 respectively

Epoch 1/20
86/86 [==============================] - 4s 28ms/step - loss: 0.5606 - accuracy: 0.8294 - val_loss: 1.2638 - val_accuracy: 0.5627
Epoch 2/20
86/86 [==============================] - 1s 11ms/step - loss: 0.3215 - accuracy: 0.8976 - val_loss: 1.1559 - val_accuracy: 0.5773
Epoch 3/20
86/86 [==============================] - 1s 7ms/step - loss: 0.2615 - accuracy: 0.9191 - val_loss: 1.0369 - val_accuracy: 0.5926
Epoch 4/20
86/86 [==============================] - 1s 6ms/step - loss: 0.2362 - accuracy: 0.9191 - val_loss: 0.9592 - val_accuracy: 0.6334
Epoch 5/20
86/86 [==============================] - 1s 7ms/step - loss: 0.2024 - accuracy: 0.9307 - val_loss: 0.8126 - val_accuracy: 0.6849
Epoch 6/20
86/86 [==============================] - 1s 8ms/step - loss: 0.1942 - accuracy: 0.9355 - val_loss: 0.7612 - val_accuracy: 0.7053
Epoch 7/20
86/86 [==============================] - 1s 6ms/step - loss: 0.1764 - accuracy: 0.9431 - val_loss: 0.7454 - val_accuracy: 0.7301
Epoch 8/20
86/86 [

Using our best model, reporting the test accuracy.

In [ ]:

newmodel = Sequential()

# Input layer takes arrays of shape (*, DIM)
newmodel.add(layers.Dense(500, activation="relu", input_shape=(DIM,)))
# Added a hidden layer
newmodel.add(layers.Dense(250, activation="relu"))
# Added a dropout layer for regularization
newmodel.add(layers.Dropout(0.2, noise_shape=None, seed=None))
# Added another hidden layer
newmodel.add(layers.Dense(200, activation="relu"))
# Output layer with NUM_CLASSES classes and softmax activation
newmodel.add(layers.Dense(NUM_CLASSES, activation="softmax"))

newmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# load the weights of your best model
newmodel.load_weights(checkpoint_path)
# measure test accuracy
test_loss, test_accuracy = newmodel.evaluate(test_X, test_y)
print("Test Accuracy:", test_accuracy)

50/50 [==============================] - 1s 7ms/step - loss: 0.6579 - accuracy: 0.7822
Test Accuracy: 0.7821720242500305
